In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,44363
2,Huelva,Confirmados PDIA 14 días,1396
3,Huelva,Tasa PDIA 14 días,"272,0346084143656"
4,Huelva,Confirmados PDIA 7 días,474
5,Huelva,Tasa PDIA 7 dias,"92,36705185416139"
6,Huelva,Total Confirmados,44561
7,Huelva,Curados,40163
8,Huelva,Fallecidos,403


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  44363.0


/tmp/ipykernel_3885/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12321.0


/tmp/ipykernel_3885/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3885/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3885/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3885/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1424 personas en los últimos 7 días 

Un positivo PCR cada 441 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,44363.0,474.0,1396.0,513170.0,92.367052,272.034608,84.0
Huelva-Costa,26204.0,288.0,893.0,289548.0,99.465374,308.411731,45.0
Condado-Campiña,13827.0,139.0,370.0,156231.0,88.970819,236.828798,35.0
Huelva (capital),12321.0,101.0,326.0,143837.0,70.218372,226.645439,23.0
Isla Cristina,3034.0,34.0,121.0,21393.0,158.930491,565.605572,8.0
San Bartolomé de la Torre,371.0,16.0,19.0,3761.0,425.418772,505.184791,8.0
Gibraleón,919.0,16.0,25.0,12737.0,125.618277,196.278559,8.0
Moguer,1912.0,23.0,59.0,21867.0,105.181323,269.812960,5.0
Sierra de Huelva-Andévalo Central,3921.0,43.0,118.0,67391.0,63.806740,175.097565,4.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cañaveral de León,29.0,3.0,8.0,396.0,757.575758,2020.202020,0.0
Almendro (El),57.0,5.0,14.0,840.0,595.238095,1666.666667,0.0
Rosal de la Frontera,71.0,6.0,21.0,1697.0,353.565115,1237.477902,0.0
Cabezas Rubias,52.0,3.0,8.0,706.0,424.929178,1133.144476,0.0
Escacena del Campo,231.0,8.0,23.0,2287.0,349.803236,1005.684303,0.0
Sanlúcar de Guadiana,34.0,3.0,3.0,403.0,744.416873,744.416873,0.0
San Silvestre de Guzmán,21.0,4.0,4.0,614.0,651.465798,651.465798,0.0
Cartaya,1851.0,41.0,117.0,20083.0,204.152766,582.582284,3.0
Isla Cristina,3034.0,34.0,121.0,21393.0,158.930491,565.605572,8.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Bollullos Par del Condado,1257.0,2.0,14.0,14387.0,13.901439,97.310072,0.0,0.142857
Niebla,356.0,2.0,13.0,4116.0,48.590865,315.840622,0.0,0.153846
San Juan del Puerto,750.0,6.0,30.0,9411.0,63.755180,318.775901,0.0,0.200000
Rociana del Condado,656.0,2.0,9.0,7939.0,25.192090,113.364404,0.0,0.222222
Almonte,2227.0,13.0,57.0,24507.0,53.046068,232.586608,2.0,0.228070
Valverde del Camino,829.0,6.0,24.0,12750.0,47.058824,188.235294,1.0,0.250000
Palos de la Frontera,989.0,3.0,12.0,11742.0,25.549310,102.197241,2.0,0.250000
Isla Cristina,3034.0,34.0,121.0,21393.0,158.930491,565.605572,8.0,0.280992
Lepe,2942.0,29.0,103.0,27880.0,104.017217,369.440459,2.0,0.281553
